In [1]:
!unzip -q '/content/drive/MyDrive/dataset_2_animation_characters.zip'

In [15]:
import cv2
import joblib
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
data_dir = pathlib.Path('/content/dataset_2_animation_characters')

In [4]:
#Calculating total number of images
len(list(data_dir.glob('*/*.jpg')))

14850

In [5]:
#Initalizing required variables
batch_size, img_height, img_width = 32, 180, 180

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 14850 files belonging to 15 classes.
Using 11880 files for training.
Found 14850 files belonging to 15 classes.
Using 2970 files for validation.


In [7]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.99818456


In [8]:
num_classes = len(train_ds.class_names)

model = keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes))

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
history = model.fit(
  train_ds,
  validation_data = val_ds,
  epochs = 10
)

Epoch 1/10
372/372 [==============================] - 29s 52ms/step - loss: 5.9626 - accuracy: 0.2274 - val_loss: 2.1501 - val_accuracy: 0.3232
Epoch 2/10
372/372 [==============================] - 20s 52ms/step - loss: 1.7112 - accuracy: 0.4531 - val_loss: 2.1178 - val_accuracy: 0.3953
Epoch 3/10
372/372 [==============================] - 19s 51ms/step - loss: 1.0366 - accuracy: 0.6715 - val_loss: 3.0102 - val_accuracy: 0.4219
Epoch 4/10
372/372 [==============================] - 19s 51ms/step - loss: 0.6176 - accuracy: 0.8112 - val_loss: 4.0791 - val_accuracy: 0.4067
Epoch 5/10
372/372 [==============================] - 19s 52ms/step - loss: 0.4108 - accuracy: 0.8775 - val_loss: 3.7565 - val_accuracy: 0.4471
Epoch 6/10
372/372 [==============================] - 19s 51ms/step - loss: 0.2976 - accuracy: 0.9129 - val_loss: 4.1201 - val_accuracy: 0.4394
Epoch 7/10
372/372 [==============================] - 19s 51ms/step - loss: 0.2872 - accuracy: 0.9180 - val_loss: 4.7403 - val_accuracy:

In [13]:
joblib.dump(model, 'model.h5')

INFO:tensorflow:Assets written to: ram://e0321fe3-fe63-4bcc-b358-d3c3f08c94c7/assets


['model.h5']

In [19]:
img = '/content/dataset_2_animation_characters/chief_wiggum/pic_0002.jpg'
img = tf.keras.utils.load_img(
    img, target_size=(img_height, img_width)
)

In [21]:
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

In [24]:
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
class_names = train_ds.class_names

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to chief_wiggum with a 100.00 percent confidence.


In [29]:
def predict_img(model, img_path):
  '''
  Input : Model and Image Path
  Output : Class Name and Confidence Score
  '''
  img = tf.keras.utils.load_img(
      img_path, target_size = (180, 180)
  )

  class_names = ['abraham_grampa_simpson', 'apu_nahasapeemapetilon',
                  'bart_simpson', 'charles_montgomery_burns', 'chief_wiggum',
                  'edna_krabappel', 'homer_simpson', 'krusty_the_clown',
                  'lenny_leonard', 'lisa_simpson', 'marge_simpson',
                  'milhouse_van_houten', 'moe_szyslak', 'ned_flanders', 'principal_skinner']

  img = tf.keras.utils.img_to_array(img)
  img = tf.expand_dims(img, 0)

  prediction = model.predict(img)
  score = tf.nn.softmax(prediction[0])

  return class_names[np.argmax(score)], 100 * np.max(score)
  

In [38]:
imgUrl = 'https://i.pinimg.com/564x/73/f5/9d/73f59debaae5e47af30f7e7e25558c16.jpg'

imgPath = tf.keras.utils.get_file('Image', origin=imgUrl)

32768/24342 [========================================] - 0s 0us/step


In [39]:
predict_img(model, imgPath)

('bart_simpson', 96.62958979606628)